# Tel Aviv Junctions - Model Training

This notebook trains ML models to predict scooter accidents at junctions.

## Overview

- Feature engineering for panel data
- Panel-aware train/test split (avoid temporal leakage)
- Baseline models (Random Forest, XGBoost)
- Evaluation and feature importance


In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    classification_report, confusion_matrix,
    roc_auc_score, roc_curve
)
from sklearn.preprocessing import StandardScaler, LabelEncoder

try:
    import xgboost as xgb
    HAS_XGBOOST = True
except ImportError:
    print("XGBoost not installed. Install with: pip install xgboost")
    HAS_XGBOOST = False

from tel_aviv_junctions.panel import load_panel_dataset
from tel_aviv_junctions.config import OUTPUT_DIR

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline


## Load Data

Load the panel dataset with accident labels.


In [ ]:
panel_path = Path(OUTPUT_DIR) / "tel_aviv_junctions_panel.csv"

if not panel_path.exists():
    raise FileNotFoundError(
        f"Panel dataset not found at {panel_path}. "
        "Please run the data extraction pipeline first."
    )

df = load_panel_dataset(str(panel_path))

print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# Check if we have accident labels
if 'accident_count' not in df.columns:
    print("\n⚠️  Warning: No 'accident_count' column found.")
    print("The dataset needs accident labels for supervised learning.")
    print("Use join_accidents_temporal() to add them.")
else:
    print(f"\n✓ Accident labels found")
    print(f"  Total accidents: {df['accident_count'].sum():,}")
    print(f"  Junction-years with accidents: {(df['accident_count'] > 0).sum():,}")

display(df.head())


## Feature Engineering

Prepare features for modeling. Handle:
- Static vs time-varying features
- Categorical encoding
- Missing values
- Feature selection


In [ ]:
def prepare_features(df):
    """Prepare features for ML model."""
    df = df.copy()
    
    # Separate target
    if 'accident_count' in df.columns:
        y = df['accident_count'].copy()
    else:
        y = None
    
    # Drop non-feature columns
    drop_cols = ['junction_id', 'osm_node_ids', 'cluster_id']
    if y is not None:
        drop_cols.append('accident_count')
    
    X = df.drop(columns=[c for c in drop_cols if c in df.columns])
    
    # Handle categorical features
    categorical_cols = ['dominant_surface', 'cycleway_type']
    for col in categorical_cols:
        if col in X.columns:
            # One-hot encode
            dummies = pd.get_dummies(X[col], prefix=col, dummy_na=True)
            X = pd.concat([X.drop(columns=[col]), dummies], axis=1)
    
    # Fill missing numeric values with median
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if X[col].isna().any():
            median_val = X[col].median()
            X[col] = X[col].fillna(median_val)
    
    return X, y

X, y = prepare_features(df)

print(f"Feature matrix shape: {X.shape}")
print(f"\nFeature columns ({len(X.columns)}):")
for i, col in enumerate(X.columns, 1):
    print(f"  {i:2d}. {col}")

if y is not None:
    print(f"\nTarget distribution:")
    print(y.describe())
    print(f"\nZero accidents: {(y == 0).sum()} ({(y == 0).mean()*100:.1f}%)")
    print(f"Non-zero accidents: {(y > 0).sum()} ({(y > 0).mean()*100:.1f}%)")


## Panel-Aware Train/Test Split

**Important**: For panel data, we should split by time to avoid data leakage.
Use earlier years for training and later years for testing.


In [ ]:
# Split by year: train on earlier years, test on later years
train_years = [2015, 2016, 2017, 2018, 2019, 2020, 2021]
test_years = [2022, 2023, 2024]

train_mask = df['year'].isin(train_years)
test_mask = df['year'].isin(test_years)

X_train = X[train_mask].copy()
X_test = X[test_mask].copy()
y_train = y[train_mask].copy() if y is not None else None
y_test = y[test_mask].copy() if y is not None else None

print(f"Training set: {len(X_train):,} rows ({train_years})")
print(f"Test set: {len(X_test):,} rows ({test_years})")

if y_train is not None:
    print(f"\nTraining target stats:")
    print(f"  Mean: {y_train.mean():.2f}")
    print(f"  Non-zero: {(y_train > 0).sum():,} ({(y_train > 0).mean()*100:.1f}%)")
    
    print(f"\nTest target stats:")
    print(f"  Mean: {y_test.mean():.2f}")
    print(f"  Non-zero: {(y_test > 0).sum():,} ({(y_test > 0).mean()*100:.1f}%)")


## Model 1: Random Forest (Regression)

Predict accident count as a continuous value.


In [ ]:
if y_train is not None:
    # Random Forest Regressor
    rf_reg = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1
    )
    
    print("Training Random Forest Regressor...")
    rf_reg.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = rf_reg.predict(X_train)
    y_test_pred = rf_reg.predict(X_test)
    
    # Metrics
    print("\nTraining Metrics:")
    print(f"  RMSE: {np.sqrt(mean_squared_error(y_train, y_train_pred)):.3f}")
    print(f"  MAE: {mean_absolute_error(y_train, y_train_pred):.3f}")
    print(f"  R²: {r2_score(y_train, y_train_pred):.3f}")
    
    print("\nTest Metrics:")
    print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred)):.3f}")
    print(f"  MAE: {mean_absolute_error(y_test, y_test_pred):.3f}")
    print(f"  R²: {r2_score(y_test, y_test_pred):.3f}")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': rf_reg.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 Most Important Features:")
    display(feature_importance.head(10))
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    top_features = feature_importance.head(15)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Importance')
    plt.title('Top 15 Feature Importances (Random Forest)')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print("No target variable available for training")


## Model 2: Random Forest (Classification)

Predict whether a junction-year will have accidents (binary classification).


In [ ]:
if y_train is not None:
    # Convert to binary: has accidents (1) or not (0)
    y_train_binary = (y_train > 0).astype(int)
    y_test_binary = (y_test > 0).astype(int)
    
    rf_clf = RandomForestClassifier(
        n_estimators=100,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'  # Handle class imbalance
    )
    
    print("Training Random Forest Classifier...")
    rf_clf.fit(X_train, y_train_binary)
    
    # Predictions
    y_train_pred_binary = rf_clf.predict(X_train)
    y_test_pred_binary = rf_clf.predict(X_test)
    y_test_proba = rf_clf.predict_proba(X_test)[:, 1]
    
    # Metrics
    print("\nTraining Metrics:")
    print(classification_report(y_train_binary, y_train_pred_binary))
    
    print("\nTest Metrics:")
    print(classification_report(y_test_binary, y_test_pred_binary))
    
    # ROC AUC
    if len(np.unique(y_test_binary)) > 1:
        auc = roc_auc_score(y_test_binary, y_test_proba)
        print(f"\nTest ROC AUC: {auc:.3f}")
        
        # Plot ROC curve
        fpr, tpr, _ = roc_curve(y_test_binary, y_test_proba)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc:.3f})')
        plt.plot([0, 1], [0, 1], 'k--', label='Random')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve - Accident Prediction')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
else:
    print("No target variable available for training")


In [ ]:
if HAS_XGBOOST and y_train is not None:
    # XGBoost Regressor
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1
    )
    
    print("Training XGBoost Regressor...")
    xgb_reg.fit(X_train, y_train)
    
    # Predictions
    y_test_pred_xgb = xgb_reg.predict(X_test)
    
    # Metrics
    print("\nTest Metrics:")
    print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred_xgb)):.3f}")
    print(f"  MAE: {mean_absolute_error(y_test, y_test_pred_xgb):.3f}")
    print(f"  R²: {r2_score(y_test, y_test_pred_xgb):.3f}")
    
    # Feature importance
    xgb_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': xgb_reg.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 Most Important Features (XGBoost):")
    display(xgb_importance.head(10))
else:
    if not HAS_XGBOOST:
        print("XGBoost not available. Install with: pip install xgboost")
    else:
        print("No target variable available for training")


## Model Comparison

Compare predictions from different models.


In [ ]:
if y_test is not None:
    comparison = pd.DataFrame({
        'actual': y_test.values,
        'rf_regression': y_test_pred,
    })
    
    if HAS_XGBOOST:
        comparison['xgb_regression'] = y_test_pred_xgb
    
    # Scatter plots
    n_models = 2 if HAS_XGBOOST else 1
    fig, axes = plt.subplots(1, n_models, figsize=(6*n_models, 5))
    if n_models == 1:
        axes = [axes]
    
    axes[0].scatter(comparison['actual'], comparison['rf_regression'], alpha=0.3)
    axes[0].plot([0, comparison['actual'].max()], [0, comparison['actual'].max()], 'r--')
    axes[0].set_xlabel('Actual Accidents')
    axes[0].set_ylabel('Predicted Accidents')
    axes[0].set_title('Random Forest')
    axes[0].grid(True, alpha=0.3)
    
    if HAS_XGBOOST:
        axes[1].scatter(comparison['actual'], comparison['xgb_regression'], alpha=0.3)
        axes[1].plot([0, comparison['actual'].max()], [0, comparison['actual'].max()], 'r--')
        axes[1].set_xlabel('Actual Accidents')
        axes[1].set_ylabel('Predicted Accidents')
        axes[1].set_title('XGBoost')
        axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nPrediction Statistics:")
    display(comparison.describe())
